1、引入库函数

In [1]:
# -*- coding: utf-8 -*-

from __future__ import division

# 引入常规库函数
import numpy as np
import pandas as pd

# 引入统计函数库
# 网上此种引入方法比较普遍
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

# 引入Kalman函数库
from pykalman import KalmanFilter

2、定义相应的自定义函数，提取对应的股票对

In [ ]:
# 股票名：get_pairs
# 输入参数：
# 1、current_date：提取数据截止的时间点
# 2、ncount：提取的样本数
# 返回值：
# pandas数组，包含交易的股票配对信息、beta、alpha、hurst、half-life、category等信息
# 功能：
# 利用聚宽的行业分类、价格信息，按照行业分类提取相应的股票对，同时比较节约时间

def get_pairs(current_date,ncount):
    # 最终的返回值数据
    ret = pd.DataFrame()
    
    # 按照聚宽的行业分类，进行相应的数据提取工作
    # 参考聚宽申万二级行业列表
    industry_list = ["HY401","HY402","HY403","HY404","HY405","HY406","HY407","HY408","HY409","HY410", \
                 "HY411","HY412","HY413","HY414","HY415","HY416","HY417","HY418","HY419","HY420", \
                 "HY421","HY422","HY423","HY424","HY425","HY426","HY427","HY428","HY429","HY432", \
                 "HY433","HY434","HY435","HY436","HY437","HY438","HY439","HY440","HY441","HY442", \
                 "HY443","HY444","HY445","HY446","HY447","HY448","HY449","HY450","HY451","HY452", \
                 "HY453","HY454","HY455","HY457","HY458","HY459","HY460","HY461","HY462","HY463", \
                 "HY464","HY465","HY466","HY467","HY468","HY469","HY470","HY471","HY472","HY473", \
                 "HY474","HY476","HY477","HY478","HY479","HY480","HY481","HY483","HY484","HY485", \
                 "HY486","HY487","HY488","HY489","HY491","HY492","HY493","HY494","HY496","HY497", \
                 "HY500","HY501","HY504","HY505","HY506","HY509","HY510","HY511","HY512","HY513", \
                 "HY514","HY515","HY516","HY517","HY518","HY519","HY520","HY521","HY523","HY524", \
                 "HY525","HY526","HY527","HY528","HY529","HY530","HY531","HY570","HY571","HY572", \
                 "HY573","HY574","HY576","HY578","HY579","HY587","HY588","HY591","HY593","HY595", \
                 "HY596","HY597","HY598","HY601"]
    
    # 证监会行业列表
    industry_list = ["A01","A02","A03","A04","A05","B06","B07","B08","B09","B10","B11", \
                 "B12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22", \
                 "C23","C24","C25","C26","C27","C28","C29","C30","C31","C32","C33", \
                 "C34","C35","C36","C37","C38","C39","C40","C41","C42","C43","D44", \
                 "D45","D46","E47","E48","E49","E50","F51","F52","G53","G54","G55", \
                 "G56","G57","G58","G59","G60","H61","H62","I63","I64","I65","J66", \
                 "J67","J68","J69","K70","L71","L72","M73","M74","M75","N76","N77", \
                 "N78","O79","O80","O81","P82","Q83","Q84","R85","R86","R87","R88","R89","S90"]
    
    # 开始进行行业循环，提取相应的
    for industry_list_i in industry_list:

        print "开始计算：%s"%str(industry_list_i)
        
        # 清空存储数据
        # 用于整理数据，中间暂存每个配对对应的pearson、beta等相关信息
        data_pd = pd.DataFrame()
        
        # 清空存储配对数据
        pairs = []
        
        # 清空数据
        stock_list = []
        
        # 获取行业列表
        stock_list = get_industry_stocks(industry_list_i, date= current_date)
        
        try:
            # 获取核心的分析数据
            # cor_data_collect是下一个核心函数
            # cor_data的数据结构：
            # 字典结构，键为（pair1，pair2）元组，值为数组[pearson_factor,adf_factor,beta,alpha,hurst,half-life]
            cor_data = cor_data_collect(stock_list,day_count = ncount,end_date = current_date)
        
        except:
            print "提取行业数据出错"
        
        
        # 剔除一部分数据不相关的数据
        # 目标为：adf检测为假的数据
        for k,v in cor_data.items():
            
            # 如果长度小于3，说明没有进行Adf等后续的检测，所以没有必要进行进一步的检查
            if len(v) > 3:
                # 增加了行业industry数据
                pairs.append([k,v,industry_list_i])
    
        # 对配对数据进行整理，合并所有数据
        data_pd = pd.concat([pd.DataFrame([p[0] for p in pairs],columns = ['p1','p2']), \
                            pd.DataFrame([p[1] for p in pairs],columns = ['pearson','adf','beta','alpha','hurst','half-life']),\
                            pd.DataFrame([p[2] for p in pairs],columns = ['category'])], \
                            axis = 1)

        # 合并数据
        # 如果需要，在合并数据的时候可以进行数据筛选
        ret = pd.concat([ret,data_pd],axis = 0)
    return ret



# 函数名：cor_data_collect
# 输入参数：
# 1、stock_list：比对的股票列表
# 2、day_count：选取的每支股票的样本数
# 3、end_date：终止时间
# 输出参数：
# ret：
# 字典结构，键为（pair1，pair2）元组，值为数组[pearson_factor,adf_factor,beta,alpha,hurst,half-life]
# 功能：
# 从所给出的stock_list中，两两配对，计算相关系数
def cor_data_collect(stock_list,day_count,end_date):
    # 获取股票列表的价格数据
    pdata = get_price(stock_list, count = day_count, end_date=end_date, frequency='daily', fields='close',fq = "pre")['close']

    # Nan数据填充为0 
    pdata = pdata.dropna(0)

    # 找到列数
    n = pdata.shape[1]

    # 找到列名
    keys = pdata.keys()

    # 最终返回的存储结果
    ret = {}
    
    # 输出处理进度
    print "行业内合计股票数为：%d"%int(n)
    
    # 逐项配对，进行匹配验证
    # 按照获取的股票数量
    for i in range(n):
        
        # 对数据进行循环比对
        for j in range(i+1, n):
            
            # 赋初值，数据为价格
            S1 = pdata[keys[i]]
            S2 = pdata[keys[j]]

            
            ################    开始进行处理，共5步          ########################
            
            # 1、赋初值,以二者股票的元组为键值
            ret[(keys[i],keys[j])] = []
            
            # 2、计算Pearson相关系数
            # calcPearson为自定义函数，见后注释
            pearson_cor = calcPearson(S1,S2)
            ret[(keys[i],keys[j])].append(pearson_cor)
            
            try:
                # 如果pearson相关系数高，则进行下一步，否则暂缓
                # 主要目的是节约时间
                if abs(pearson_cor) > 0.6:
                    
                    # 3、计算相关CADF数据
                    # 初值均赋值为最小项
                    adf_factor = False
                    beta = 0
                    alpha = 0 

                    # Cadf_test为自定义函数，见后注释
                    adf_factor,beta,alpha = Cadf_test(keys[i],keys[j],day_count,end_date)

                    # 节约运算时间，如果的确是均值回归，再进行计算
                    if adf_factor:
                        
                        # 数组中追加述职
                        ret[(keys[i],keys[j])].append(adf_factor)
                        ret[(keys[i],keys[j])].append(beta)
                        ret[(keys[i],keys[j])].append(alpha)

                        # 4、计算hurst
                        # 需要先计算两者的残差
                        residual = pd.Series(S1 - beta * S2 - alpha)

                        #  数据为负会出错，所以加100
                        # hurst为自定义函数，见后注释
                        ret[(keys[i],keys[j])].append(hurst(residual+100))

                        # 5、计算half-life
                        # half_life为自定义函数，见后注释
                        ret[(keys[i],keys[j])].append(half_life(residual+100))
            except Exception, e: 
                print "出错：%s"%str(e)
                continue
    return ret

# 函数名：Cadf_test
# 输入参数：
# 1、sec1：比对的第一支股票
# 2、sec2：比对的第二只股票
# 3、ncount：样本数
# 4、end_date：终止时间
# 输出参数：
# 序列：
# Adf_factor,beta,alpha：分别表示Adf验证是否通过，kalman方程中的beta，kalman方程中的alpha
# 功能：
# 从给出的两个股票中，计算adf_factor、基于kalman方程的beta、基于的kalman方程的alpha
def Cadf_test(sec1,sec2,ncount,end_date):
    # 将股票改造成数组，并对参数赋初值
    secs = [sec1, sec2]
    ncount = ncount
    end_date = end_date
    
    # 获得价格数据
    p_data = get_price(secs, count =  ncount, end_date= end_date, frequency='1d', fields='close',fq = "pre")['close']

    # 基于kalman方程获得alpha、beta
    beta_kf = kalman_beta(sec1 = sec1 ,sec2 = sec2,count = ncount, end_date = end_date)
    beta = beta_kf[0]
    alpha = beta_kf[1]

    # 利用alpha、beta计算两个股票的残差
    # 注意：
    # 1、股票1与股票2的顺序，sec2 = sec1 * beta + alpha
    # 2、beta、alpha均包含在beta_kf中
    price_pd = pd.DataFrame(p_data[secs[1]]- np.dot(sm.add_constant(p_data[secs[0]], prepend=False), beta_kf))
    price_pd.columns = ['res']
    
    # 使用adf计算adf的值
    cadf = ts.adfuller(price_pd["res"])
    
    # 如果小于阈值，则可以以5%的置信度可以推翻Null hypothesis
    # null hypothesis:  there isn't a cointegrating relationship at the 5% level.
    if cadf[1] < 0.01:
        return True,beta,alpha
    else:
        return False,beta,alpha

    
# 函数名：hurst
# 输入参数：
# 1、ts：残差序列
# 输出参数：
# 大于0.5则说明是趋势，小于0.5均值回复
def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0


#计算Pearson系数
def calcPearson(x,y):
    #计算特征和类的平均值
    def _calcMean(x,y):
        sum_x = sum(x)
        sum_y = sum(y)
        
        n = len(x)
        if n != 0:
            x_mean = float(sum_x+0.0)/n
            y_mean = float(sum_y+0.0)/n
            return x_mean,y_mean
        else:
            return 1,1
    x_mean,y_mean = _calcMean(x,y)	#计算x,y向量平均值
    n = len(x)
    sumTop = 0.0
    sumBottom = 0.0
    x_pow = 0.0
    y_pow = 0.0
    for i in range(n):
        sumTop += (x[i]-x_mean)*(y[i]-y_mean)
    for i in range(n):
        x_pow += math.pow(x[i]-x_mean,2)
    for i in range(n):
        y_pow += math.pow(y[i]-y_mean,2)
    sumBottom = math.sqrt(x_pow*y_pow)
    if sumBottom != 0:
        p = sumTop/sumBottom
        return p
    else:
        return 0

# 计算P半衰期
# 看是否能在半衰期内均值回复
def half_life(time_series):
    
    # np.roll的功能如下：
    # >>> x = np.arange(10)
    # >>> np.roll(x, 2)
    # array([8, 9, 0, 1, 2, 3, 4, 5, 6, 7])
    lag = np.roll(time_series, 1)
    
    # 第一个置为0 
    lag[0] = 0
    
    # 去中间的差，并且第一个设置为0
    ret = time_series - lag
    ret[0] = 0

    # lag 作为自变量
    # adds intercept terms to X variable for regression
    lag2 = sm.add_constant(lag)

    # 线性拟合，确定beta
    model = sm.OLS(ret, lag2)
    res = model.fit()

    half_life = -np.log(2) / res.params[1]
    
    return half_life


# 函数名：kalman_beta
# 输入参数：
# 1、sec1：比对的第一支股票
# 2、sec2：比对的第二只股票
# 3、ncount：样本数
# 4、end_date：终止时间
# 输出参数：
# state_means，同时包含了beta、alpha
# 第1个参数为beta
# 第2个参数为alpha
def kalman_beta(sec1 = '000858.XSHE' ,sec2 = '000300.XSHG',count = 400, end_date = '2015-3-1'):
    # 赋初值
    secs = [sec1, sec2]
    ncount = count
    end_date = end_date
    
    # 获取价格数据
    data = get_price(secs, count =  ncount, end_date= end_date, frequency='1d', fields='close',fq = "pre")['close']
    data.index.name = 'Date'
    
    # 观察矩阵
    # 注意：
    # 1、观察到的是sec1数据，sec1是自变量x，sec2是因变量y
    # 2、需要使用add_constant来模拟alpha
    # 3、需要使用np.newaxis来增加维度
    obs_mat = sm.add_constant(data[secs[0]].values, prepend=False)[:, np.newaxis]

    kf = KalmanFilter(n_dim_obs=1, n_dim_state=2, # y is 1-dimensional, (alpha, beta) is 2-dimensional
                  initial_state_mean=np.ones(2),
                  initial_state_covariance=np.ones((2, 2)),
                  transition_matrices=np.eye(2),  # 不发生变化，都是单位矩阵
                  observation_matrices=obs_mat,   # 观察矩阵
                  observation_covariance=10**2,
                  transition_covariance=0.01**2 * np.eye(2))
    
    # 相当于使用sec2来进行训练，模拟出beta、alpha
    state_means, state_covs = kf.filter(data[secs[1]][:, np.newaxis])
    return state_means[-1]

#6 R/S算法计算Hurst
#变量：x-日收益率-list(之后使用np.array变换成数组) 
#输出：计算得到的Hurst值
def hurst_joinquant(X):
    
    #输入日回报率
    X = np.array(X)
    
    #N代表最大的片段值（即不对X分割时）
    N = X.size
    
    T = np.arange(1, N + 1)
    Y = np.cumsum(X)
    
    #分别计算不同长度的片段的均值
    Ave_T = Y / T

    #每个片段的最大差距R_T
    R_T = np.zeros(N)
    #对应的每段的标准差S_T
    S_T = np.zeros(N)
    
    #分别对不同的大小的切片计算R_T和S_T
    for i in range(N):
        S_T[i] = np.std(X[:i + 1])
        Z_T = Y - T * Ave_T[i]
        R_T[i] = np.ptp(Z_T[:i + 1])
        
    #计算R/S
    R_S = R_T / S_T
    
    #将lg(R/S)作为被解释变量Y
    R_S = np.log(R_S)[1:]
    
    #将lgt作为解释变量X
    n = np.log(T)[1:]
    A = np.column_stack((n, np.ones(n.size)))
    
    #回归得到的斜率即为hurst指数
    [m, c] = np.linalg.lstsq(A, R_S)[0]
    H = m
    return H

3、使用上面定义的函数进行配对数据提取

并存入文件中

In [ ]:
pairs_pd = get_pairs("2018-6-10",200)

# 确定文件名
file_name = "kalman_pairs.csv"

# 写入文件
write_file(file_name, pairs_pd.to_csv(), append=False) #写到文件中

In [69]:
pairs_pd

,p1,p2,pearson,adf,beta,alpha,hurst,half-life,category
0,600096.XSHG,603585.XSHG,0.868464,True,3.101326,3.08546,0.227525,31.704278,C13
0,600096.XSHG,603585.XSHG,0.868464,True,3.101326,3.08546,0.227525,31.704278,C14
